# **Merge Train Meteo + Google Trends**

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot') 

import warnings 
warnings.filterwarnings ('ignore')

In [ ]:
train_meteo = pd.read_csv("../data/processed/train_meteo.csv")
train_google = pd.read_csv("../data/processed/train.csv")
test_meteo = pd.read_csv("../data/processed/test_meteo.csv")
test_google = pd.read_csv("../data/processed/test.csv")

print("Train meteo shape:", train_meteo.shape)
print("Train google shape:", train_google.shape)

Train meteo shape: (9196, 14)
Train google shape: (9196, 13)


In [ ]:
train_meteo.head()

,Id,week,region_code,region_name,year,week_num,date,month,t,pmer,ff,rr24,u
0,3235,201352,42,ALSACE,2013,52,2013-12-23,12,276.327419,102422.278226,2.422984,117.2,84.774194
1,3236,201352,72,AQUITAINE,2013,52,2013-12-23,12,279.329657,102291.177730,2.540899,129.0,85.496788
2,3237,201352,83,AUVERGNE,2013,52,2013-12-23,12,276.725770,102346.066946,2.853799,51.4,74.498973
3,3238,201352,25,BASSE-NORMANDIE,2013,52,2013-12-23,12,280.374394,101897.250674,5.680628,217.7,84.512129
4,3239,201352,26,BOURGOGNE,2013,52,2013-12-23,12,275.866599,102446.518219,2.363968,43.3,88.607287


In [4]:
train_google.head()

,Id,week,region_name,TauxGrippe,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,requete_grippe,requete_grippe_aviaire_vaccin,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie
0,5523,201152,ALSACE,66,448112,479905,526481,250849,146978,1852325,2,2,2
1,5524,201152,AQUITAINE,24,732120,765143,896030,516065,344875,3254233,2,2,2
2,5525,201152,AUVERGNE,91,294709,305778,375485,223830,150880,1350682,3,3,2
3,5526,201152,BASSE-NORMANDIE,49,354514,336201,401766,228888,154315,1475684,2,2,2
4,5527,201152,BOURGOGNE,33,372023,369737,450664,269109,181201,1642734,2,2,1


In [5]:
train_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9196 entries, 0 to 9195
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           9196 non-null   int64  
 1   week         9196 non-null   int64  
 2   region_code  9196 non-null   int64  
 3   region_name  9196 non-null   object 
 4   TauxGrippe   9196 non-null   int64  
 5   year         9196 non-null   int64  
 6   week_num     9196 non-null   int64  
 7   date         9196 non-null   object 
 8   month        9196 non-null   int64  
 9   t            9196 non-null   float64
 10  pmer         9196 non-null   float64
 11  ff           9196 non-null   float64
 12  rr24         9196 non-null   float64
 13  u            9196 non-null   float64
dtypes: float64(5), int64(7), object(2)
memory usage: 1005.9+ KB


In [6]:
train_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9196 entries, 0 to 9195
Data columns (total 13 columns):
 #   Column                                                                              Non-Null Count  Dtype 
---  ------                                                                              --------------  ----- 
 0   Id                                                                                  9196 non-null   int64 
 1   week                                                                                9196 non-null   int64 
 2   region_name                                                                         9196 non-null   object
 3   TauxGrippe                                                                          9196 non-null   int64 
 4   pop_0_19                                                                            9196 non-null   int64 
 5   pop_20_39                                                                           9196 non-null   int6

## Séléction des colonnes

In [7]:
cols_meteo = [
    "Id",
    "week",
    "region_code",
    "region_name",
    "TauxGrippe",
    "year",
    "week_num",
    "month",
    "t",
    "pmer",
    "ff",
    "rr24",
    "u"
]

train_meteo = train_meteo[cols_meteo]

In [25]:
cols_meteo_test= [
    "Id",
    "week",
    "region_code",
    "region_name",
    "year",
    "week_num",
    "month",
    "t",
    "pmer",
    "ff",
    "rr24",
    "u"
]

test_meteo = test_meteo[cols_meteo_test]

In [10]:
cols_google = [
    "Id",
    "week",
    "region_name",
    "TauxGrippe",
    "pop_0_19",
    "pop_20_39",
    "pop_40_59",
    "pop_60_74",
    "pop_75_plus",
    "pop_total",
    "requete_grippe",
    "requete_grippe_aviaire_vaccin",
    "requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie"
]

train_google = train_google[cols_google]

In [26]:
cols_google_test = [
    "Id",
    "week",
    "region_name",
    "pop_0_19",
    "pop_20_39",
    "pop_40_59",
    "pop_60_74",
    "pop_75_plus",
    "pop_total",
    "requete_grippe",
    "requete_grippe_aviaire_vaccin",
    "requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie"
]

test_google = test_google[cols_google_test]

In [17]:
train_full = train_meteo.merge(
    train_google,
    on=["region_name", "week"],
    how="inner",
    suffixes=("", "_google")
)

print(train_full.shape)

(9196, 24)


In [27]:
test_full = test_meteo.merge(
    test_google,
    on=["region_name", "week"],
    how="inner",
    suffixes=("", "_google")
)

print(test_full.shape)

(2288, 22)


In [18]:
train_full.head()

,Id,week,region_code,region_name,TauxGrippe,year,week_num,month,t,pmer,...,TauxGrippe_google,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,requete_grippe,requete_grippe_aviaire_vaccin,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie
0,5523,201152,42,ALSACE,66,2011,52,12,278.613306,101860.544355,...,66,448112,479905,526481,250849,146978,1852325,2,2,2
1,5524,201152,72,AQUITAINE,24,2011,52,12,282.006707,102447.682927,...,24,732120,765143,896030,516065,344875,3254233,2,2,2
2,5525,201152,83,AUVERGNE,91,2011,52,12,278.906452,102179.798387,...,91,294709,305778,375485,223830,150880,1350682,3,3,2
3,5526,201152,25,BASSE-NORMANDIE,49,2011,52,12,281.327209,101840.124031,...,49,354514,336201,401766,228888,154315,1475684,2,2,2
4,5527,201152,26,BOURGOGNE,33,2011,52,12,278.612857,102035.428571,...,33,372023,369737,450664,269109,181201,1642734,2,2,1


In [19]:
train_full.to_csv("train_full.csv", index=False)

In [28]:
test_full.to_csv("test_full.csv", index=False)